In [1]:
import pandas as pd
import numpy as np
import re
import json
import html
# import emot
import string

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import words
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('words')
# nltk.download('wordnet')
nltk.download('stopwords')
tkn = TweetTokenizer()
exclude = set(string.punctuation)
StopWords = stopwords.words("english")
bag = words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\5115100178\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\5115100178\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path_to_dataset = "C:/xampp/htdocs/Data-Mining/FP/dataset/"
path_to_clean = "C:/xampp/htdocs/Data-Mining/FP/clean/"
filename = "SS-Twitter"
# filename = "STS-Gold" #
# filename = "STS-Test"

path_to_experiment = path_to_clean + filename +"_EXPERIMENT.csv"

In [3]:
# # #Loading dataset 
# # Labels = ["class", "id", "date", "query", "username","tweet"] #STS
# # Labels = ["id","polarity","tweet"] #Gold
Labels = ["mean pos","mean neg","tweet"] #SS

# # data = pd.read_csv(path_to_dataset + filename +".csv",names=Labels,sep=";",header=0) #Gold
# # data = pd.read_csv(path_to_dataset + filename +".csv",names=Labels) #STS
# data = pd.read_csv(path_to_dataset + filename+".csv") #SS
data = pd.read_csv(path_to_experiment) #EXPERIMENT


# # data = data[["id", "date", "query", "username","tweet","class"]] #STS-Test
# # data = data[['id', 'tweet', 'polarity']] #STS-Gold

data.head()

,mean pos,mean neg,Tweet,Clean,class,NO URL,NO STOPWORD,NO NUMBER,NO REPEAT,NO ACRONYM,NO NEGATION,OUR
0,3,2,?RT @justinbiebcr: The bigger the better....if...,rt @justinbiebcr bigger better ... know mean ;),4,rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr the bigger the better ... if ...,rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt bigger better ... know mean happy
1,3,1,"Listening to the ""New Age"" station on @Slacker...",listening new age station @slackeradio,4,listening new age station @slackeradio htp://s...,listening to the new age station on @slackeradio,listening new age station @slackeradio,listening new age station @slackerradio,listening new age station @slackeradio,listening new age station @slackeradio,listening new age station
2,1,1,I favorited a YouTube video -- Drake and Josh ...,favorited youtube video drake josh storm rock,2,favorited youtube video drake josh storm rock ...,i favorited a youtube video drake and josh the...,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock
3,4,2,i didnt mean knee high I ment in lengt it goes...,did not mean knee high ment lengt goes knes cu...,4,did not mean knee high ment lengt goes knes cu...,i did not mean knee high i ment in lengt it go...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knees c...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knes cu...
4,2,1,I wana see the vid Kyan,wanna see video kyan,4,wanna see video kyan,i wanna see the video kyan,wanna see video kyan,wanna see video kyan,wana see vid kyan,wanna see video kyan,wanna see video kyan


In [5]:
# ## CEK Bentuk data

# # Cek missing value

# print('\nNumber of missing values:')
# for col in data.columns:
#     print('\t%s: %d' % (col, data[col].isnull().sum()))
    
# # cek jumlah perkelas
# print('\nNumber of class values:')
# print(data.iloc[:,-1].value_counts(dropna=False))

In [4]:
#### PREPROCESSING

## Tokenizing
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# #Repeated Word
def normalize_repeated_char(token):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match = r'\1\2\3'
    def replace(old_word):
        if old_word in bag:
#         ps = PorterStemmer()
#         ps.stem(old_word)
#         if wordnet.synsets(old_word):
#         if old_word in words.words():
            return old_word
        new_word = repeat_pattern.sub(match, old_word)
        return replace(new_word) if new_word != old_word else new_word
    
    correct = [replace(word) for word in token]
    return correct

# #Negation
def normalized_negation_phrase(tokenized_sentence):

    mydict = { "did not": ["didn't", "didnt"], 
              "do not": ["don't","dont"], 
              "can not": ["can't","cant"], 
              "sould not": ["souldn't","souldnt"],
              "would not": ["wouldn't","wouldnt"], 
              "could not": ["couldn't","couldnt"],
              "have not": ["haven't","havent"],
              "had not": ["hadn't","hadnt"],
             }
    
    for index in range(len(tokenized_sentence)):
        for key, value in mydict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# #Stopword
# def stopword(tokenized_sentence):
# #     StopWords = stopwords.words("english")
#     StopWords = stopwords.words("english")
# #     token = nltk.word_tokenize(contoh)
#     for v in tokenized_sentence:
#         if v not in StopWords:
#             return ' '.join(tokenized_sentence)
# #             print(v)

    

# #Akronim
def normalize_slang_words(tokenized_sentence):
    slang_word_dict = json.loads(open("slang_word_dict.txt", 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# #Emoticon
def emoticon(tokenized_sentence):
    emoticon_dict = json.loads(open("emoticon_dict.txt", 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in emoticon_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# Join
# def join_token(tokenized_sentence):
#     for index in range(len(tokenized_sentence)):
#         tokenized_sentence[index] = '|'.join(str(i) for i in tokenized_sentence[index])
#     return ' '.join(tokenized_sentence)

In [35]:
def preprocessing(comment):
    
    comment = comment.lower() #lower
    comment = html.unescape(comment) #html tag
    comment = re.sub("@[A-Za-z0-9]+", "", comment) #no Mention
    comment = re.sub("#[A-Za-z0-9]+", "", comment) #no Hashtag
    comment = re.sub("[0-9]", "", comment) #no Number
    comment = re.sub(r"http\S+", "", comment) #no HTML    
    comment = tkn.tokenize(comment)#token
    comment = emoticon(comment) #Emoticon
    comment = normalize_repeated_char(comment) #Repeated
    comment = normalized_negation_phrase(comment) #Negation
    comment = normalize_slang_words(comment) #slang
    comment = [word for word in comment if word not in StopWords] #Stopword
    comment = ' '.join(ch for ch in comment if ch not in exclude) #Puntuation
    
    return comment

In [36]:
data['OUR'] = data['Tweet'].apply(preprocessing)
data.head()

,mean pos,mean neg,Tweet,Clean,class,NO URL,NO STOPWORD,NO NUMBER,NO REPEAT,NO ACRONYM,NO NEGATION,OUR
0,3,2,?RT @justinbiebcr: The bigger the better....if...,rt @justinbiebcr bigger better ... know mean ;),4,rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr the bigger the better ... if ...,rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt @justinbiebcr bigger better ... know mean ;),rt bigger better ... know mean happy
1,3,1,"Listening to the ""New Age"" station on @Slacker...",listening new age station @slackeradio,4,listening new age station @slackeradio htp://s...,listening to the new age station on @slackeradio,listening new age station @slackeradio,listening new age station @slackerradio,listening new age station @slackeradio,listening new age station @slackeradio,listening new age station
2,1,1,I favorited a YouTube video -- Drake and Josh ...,favorited youtube video drake josh storm rock,2,favorited youtube video drake josh storm rock ...,i favorited a youtube video drake and josh the...,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock,favorited youtube video drake josh storm rock
3,4,2,i didnt mean knee high I ment in lengt it goes...,did not mean knee high ment lengt goes knes cu...,4,did not mean knee high ment lengt goes knes cu...,i did not mean knee high i ment in lengt it go...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knees c...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knes cu...,did not mean knee high ment lengt goes knes cu...
4,2,1,I wana see the vid Kyan,wanna see video kyan,4,wanna see video kyan,i wanna see the video kyan,wanna see video kyan,wanna see video kyan,wana see vid kyan,wanna see video kyan,wanna see video kyan


In [37]:
# ## Save Clean Data

# data.to_csv(path_to_clean+filename+"_BASELINE.csv",index=False) 
# # #with url, stopword, number, repeated, acronym, negation

data.to_csv(path_to_clean+filename+"_EXPERIMENT.csv",index=False) 
# # #combination of url, stopword, number, repeated, acronym, negation, our preprocessing

In [69]:
# # #Make Dict

# # #Join
# file1 = pd.read_csv(path_to_clean + "STS-Test.csv")
# file2 = pd.read_csv(path_to_clean + "SS-Twitter.csv")
# file3 = pd.read_csv(path_to_clean + "STS-Gold.csv")

# s1 = file1['Clean']
# s2 = file2['Clean']
# s3 = file3['Clean']

# tweet = list()
# tweet = s1.append(s2)
# tweet = tweet.append(s3)
# # tweet.reset_index()


In [126]:
# # #Save
# tweet.to_csv(path_to_clean+"All_Tweet.csv",index=False)

In [127]:
# all_tweet = pd.read_csv(path_to_clean + "All_Tweet.csv",header=None)
# all_tweet.head()

In [128]:
# # # Emoticon Dict

# for index, row in all_tweet.iterrows():
#     ContentnoMention = re.sub("[A-Za-z]","",str(row))

#     print(ContentnoMention)

In [155]:
# # #Check kata baku
# def is_english_word(comment):
#     # creation of this dictionary would be done outside of 
#     #     the function because you only need to do it once.
#     comment = comment.lower() #lower
#     comment = html.unescape(comment) #html tag
#     comment = re.sub("@[A-Za-z0-9]+", "", comment) #no Mention
#     comment = re.sub("#[A-Za-z0-9]+", "", comment) #no Hashtag
#     comment = re.sub("[0-9]", "", comment) #no Number
#     comment = re.sub(r"http\S+", "", comment) #no HTML    
#     comment = tkn.tokenize(comment)#token
#     dictionary = dict.fromkeys(words.words(), None)
#     for i,v in enumerate(comment):
#         try:
#             x = dictionary[v]
#             continue
#         except KeyError:
#             print (v)

In [154]:
# data['Tweet'].apply(is_english_word)

In [112]:
# # Scrapping

# from bs4 import BeautifulSoup
# import requests, json
# resp = requests.get('http://www.netlingo.com/acronyms.php')
# soup = BeautifulSoup(resp.text, "html.parser")
# slangdict= {}
# key=""
# value=""
# for div in soup.findAll('div', attrs={'class':'list_box3'}):
#     for li in div.findAll('li'):
#         for a in li.findAll('a'):
#             key =a.text
#         value = li.text.split(key)[1]
#         slangdict[key]=value
# with open('C:/Users/USER/Downloads/tegar nitip/SMT 8/DatMin/FP/myslang.json', 'w') as find:
#     json.dump(slangdict,find,indent=2)

In [8]:
# Convert to csv

# data = pd.read_csv(path_to_dataset + filename, delimiter="\t", encoding='latin1')
# data.to_csv(path_to_dataset+"SS-Twitter.csv",index=False)
# # f = open(path_to_dataset + filename, "r")
# # content = f.read()

In [ ]:
# path_to_clean = "C:/xampp/htdocs/Data-Mining/FP/dataset/"

# Class = list()
# pos = base['mean pos'].to_list()
# neg = base['mean neg'].to_list()

# for i, val in zip(pos,neg):
#     senti = i - val
#     if senti < 0:
#         Class.append(0)
#     elif senti > 0:
#         Class.append(4)
#     else:
#         Class.append(2)

# base['class'] = Class
# base.head()

# base.to_csv(path_to_clean+filename+".csv",index=False)

['holiday', 'time', 'to']